In [33]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

In [6]:
for folder in os.listdir("Dataset"):
    print(folder, len(os.listdir(os.path.join("Dataset", folder))))

glioma_tumor 926
meningioma_tumor 937
no_tumor 500
pituitary_tumor 901


In [10]:
target_folder = "Dataset/meningioma_tumor"  # change this to your folder
base_name = "mm"

# Get all image files
images = [f for f in os.listdir(target_folder) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.gif'))]
images.sort()

existing_names = set(os.listdir(target_folder))  # To check existing files

for i, filename in enumerate(images, start=1):
    ext = os.path.splitext(filename)[1]
    new_name = f"{base_name} ({i}){ext}"
    src = os.path.join(target_folder, filename)
    dst = os.path.join(target_folder, new_name)

    # Avoid overwriting existing files
    if new_name in existing_names:
        print(f"⚠️ Skipped: {new_name} already exists.")
        continue

    os.rename(src, dst)
    print(f"Renamed: {filename} → {new_name}")

print("✅ All images in", target_folder, "renamed safely!")


Renamed: image(100).jpg → mm (1).jpg
Renamed: image(12).jpg → mm (2).jpg
Renamed: image(120).jpg → mm (3).jpg
Renamed: image(121).jpg → mm (4).jpg
Renamed: image(123).jpg → mm (5).jpg
Renamed: image(124).jpg → mm (6).jpg
Renamed: image(125).jpg → mm (7).jpg
Renamed: image(126).jpg → mm (8).jpg
Renamed: image(127).jpg → mm (9).jpg
Renamed: image(13).jpg → mm (10).jpg
Renamed: image(9).jpg → mm (11).jpg
Renamed: image(90).jpg → mm (12).jpg
Renamed: image(91).jpg → mm (13).jpg
Renamed: image(92).jpg → mm (14).jpg
Renamed: image(93).jpg → mm (15).jpg
Renamed: image(94).jpg → mm (16).jpg
Renamed: image(95).jpg → mm (17).jpg
Renamed: image(96).jpg → mm (18).jpg
Renamed: image(97).jpg → mm (19).jpg
Renamed: image(98).jpg → mm (20).jpg
Renamed: image(99).jpg → mm (21).jpg
Renamed: image.jpg → mm (22).jpg
Renamed: m  (205).jpg → mm (23).jpg
Renamed: m (1).jpg → mm (24).jpg
Renamed: m (10).jpg → mm (25).jpg
Renamed: m (100).jpg → mm (26).jpg
Renamed: m (101).jpg → mm (27).jpg
Renamed: m (102).jp

In [11]:
for folder in os.listdir("Dataset"):
    print(folder, len(os.listdir(os.path.join("Dataset", folder))))

glioma_tumor 926
meningioma_tumor 937
no_tumor 500
pituitary_tumor 901


In [12]:
import os
import shutil
import random

# Paths
SOURCE_DIR = "Dataset"
TARGET_DIR = "Dataset_split"

# Split ratios
TRAIN_RATIO = 0.7
VAL_RATIO = 0.15
TEST_RATIO = 0.15

# Create target directories
for split in ["train", "val", "test"]:
    os.makedirs(os.path.join(TARGET_DIR, split), exist_ok=True)

# Loop through each class
for class_name in os.listdir(SOURCE_DIR):
    class_path = os.path.join(SOURCE_DIR, class_name)
    if not os.path.isdir(class_path):
        continue

    images = os.listdir(class_path)
    random.shuffle(images)

    total = len(images)
    train_end = int(TRAIN_RATIO * total)
    val_end = int((TRAIN_RATIO + VAL_RATIO) * total)

    train_imgs = images[:train_end]
    val_imgs = images[train_end:val_end]
    test_imgs = images[val_end:]

    # Create class subfolders in each split
    for split_name, split_imgs in zip(["train", "val", "test"], [train_imgs, val_imgs, test_imgs]):
        split_class_dir = os.path.join(TARGET_DIR, split_name, class_name)
        os.makedirs(split_class_dir, exist_ok=True)

        for img in split_imgs:
            src = os.path.join(class_path, img)
            dst = os.path.join(split_class_dir, img)
            shutil.copy(src, dst)

print(" Dataset successfully split into train, val, and test!")


✅ Dataset successfully split into train, val, and test!


In [15]:
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

train_dataset = tf.keras.utils.image_dataset_from_directory(
    "dataset_split/train",
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

val_dataset = tf.keras.utils.image_dataset_from_directory(
    "dataset_split/val",
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

test_dataset = tf.keras.utils.image_dataset_from_directory(
    "dataset_split/test",
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

Found 2283 files belonging to 4 classes.
Found 490 files belonging to 4 classes.
Found 491 files belonging to 4 classes.


In [16]:
AUTOTUNE = tf.data.AUTOTUNE

def normalize(image, label):
    return tf.cast(image, tf.float32) / 255.0, label

train_dataset = train_dataset.map(normalize).cache().shuffle(1000).prefetch(AUTOTUNE)
val_dataset = val_dataset.map(normalize).cache().prefetch(AUTOTUNE)
test_dataset = test_dataset.map(normalize).cache().prefetch(AUTOTUNE)


In [27]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
for layer in base_model.layers:
    layer.trainable = False  # Freeze base layers

# Add custom classifier
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(4, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Compile
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [28]:
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
lr_schedule = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3)
checkpoint = ModelCheckpoint("best_model.keras", monitor='val_loss', save_best_only=True)


In [29]:
history = model.fit(     
    train_dataset,
    validation_data=val_dataset,
    epochs=30,
    callbacks=[early_stop, lr_schedule, checkpoint]
)

Epoch 1/30
72/72 ━━━━━━━━━━━━━━━━━━━━ 587s 8s/step - accuracy: 0.3163 - loss: 1.4087 - val_accuracy: 0.4388 - val_loss: 1.2801 - learning_rate: 1.0000e-04
Epoch 2/30
72/72 ━━━━━━━━━━━━━━━━━━━━ 586s 8s/step - accuracy: 0.3653 - loss: 1.3255 - val_accuracy: 0.5449 - val_loss: 1.2227 - learning_rate: 1.0000e-04
Epoch 3/30
72/72 ━━━━━━━━━━━━━━━━━━━━ 587s 8s/step - accuracy: 0.4266 - loss: 1.2598 - val_accuracy: 0.5694 - val_loss: 1.1809 - learning_rate: 1.0000e-04
Epoch 4/30
72/72 ━━━━━━━━━━━━━━━━━━━━ 584s 8s/step - accuracy: 0.4590 - loss: 1.2114 - val_accuracy: 0.5653 - val_loss: 1.1431 - learning_rate: 1.0000e-04
Epoch 5/30
72/72 ━━━━━━━━━━━━━━━━━━━━ 590s 8s/step - accuracy: 0.4976 - loss: 1.1500 - val_accuracy: 0.5857 - val_loss: 1.1082 - learning_rate: 1.0000e-04
Epoch 6/30
72/72 ━━━━━━━━━━━━━━━━━━━━ 585s 8s/step - accuracy: 0.5633 - loss: 1.1054 - val_accuracy: 0.6224 - val_loss: 1.0746 - learning_rate: 1.0000e-04
Epoch 7/30
72/72 ━━━━━━━━━━━━━━━━━━━━ 611s 9s/step - accuracy: 0.5760 

In [30]:
for layer in base_model.layers[-4:]:
    layer.trainable = True

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)
history_finetune = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=20,
    callbacks=[early_stop, lr_schedule, checkpoint]
)

Epoch 1/20
72/72 ━━━━━━━━━━━━━━━━━━━━ 570s 8s/step - accuracy: 0.7823 - loss: 0.6151 - val_accuracy: 0.8184 - val_loss: 0.5299 - learning_rate: 1.0000e-05
Epoch 2/20
72/72 ━━━━━━━━━━━━━━━━━━━━ 592s 8s/step - accuracy: 0.8283 - loss: 0.4789 - val_accuracy: 0.8184 - val_loss: 0.4845 - learning_rate: 1.0000e-05
Epoch 3/20
72/72 ━━━━━━━━━━━━━━━━━━━━ 563s 8s/step - accuracy: 0.8559 - loss: 0.4166 - val_accuracy: 0.8612 - val_loss: 0.3997 - learning_rate: 1.0000e-05
Epoch 4/20
72/72 ━━━━━━━━━━━━━━━━━━━━ 574s 8s/step - accuracy: 0.8804 - loss: 0.3521 - val_accuracy: 0.8694 - val_loss: 0.3637 - learning_rate: 1.0000e-05
Epoch 5/20
72/72 ━━━━━━━━━━━━━━━━━━━━ 583s 8s/step - accuracy: 0.8909 - loss: 0.3040 - val_accuracy: 0.8878 - val_loss: 0.3390 - learning_rate: 1.0000e-05
Epoch 6/20
72/72 ━━━━━━━━━━━━━━━━━━━━ 581s 8s/step - accuracy: 0.9163 - loss: 0.2624 - val_accuracy: 0.8857 - val_loss: 0.3112 - learning_rate: 1.0000e-05
Epoch 7/20
72/72 ━━━━━━━━━━━━━━━━━━━━ 581s 8s/step - accuracy: 0.9229 

In [31]:
test_loss, test_acc = model.evaluate(test_dataset)
print(f"\n Test Accuracy: {test_acc:.4f}")
print(f" Test Loss: {test_loss:.4f}")

16/16 ━━━━━━━━━━━━━━━━━━━━ 77s 5s/step - accuracy: 0.9206 - loss: 0.2097

 Test Accuracy: 0.9206
 Test Loss: 0.2097


In [32]:
model.save("brain_tumor_vgg16.keras")
print("Model saved successfully!")

Model saved successfully!
